# Convolutional CRFs for Semantic Segmentation

### This notebook contains code and results of experiments on the ICLR Paper submisson "Convolutional CRFs for Semantic Segmentation" as part of the ICLR 2019 Reproducibility challenge

### Load Images

In [38]:
import imageio
import matplotlib.pyplot as plt
import numpy as np
import scipy
import skimage.transform

IMG = 'data/img_bicycle.png'
LABELS = 'data/img_bicycle_labels.png'

image = imageio.imread(IMG)
labels = imageio.imread(LABELS)

### Produce unary by adding noise to label

In [33]:
num_classes = 21
keep_prop=0.8
scale=8

#output
unary = None

shape = labels.shape # (H, W)
labels = labels.reshape(shape[0], shape[1]) # H * W

# Onehot encoding of labels
onehot = np.eye(num_classes)[labels] # H * W * num_classes


lower_shape =  (shape[0] // scale, shape[1] // scale)

# Scale down onehot labels to 1/8
label_down = skimage.transform.resize(onehot, 
                                      (lower_shape[0], lower_shape[1], num_classes), 
                                      order=1, 
                                      preserve_range=True, 
                                      mode='constant') # (lower_shape[0], lower_shape[1], num_classes)

# scale up onehot of labels to original
onehot_up = skimage.transform.resize(label_down,
                                      (shape[0], shape[1], num_classes),
                                      order=1, preserve_range=True,
                                      mode='constant')


noise = np.random.randint(0, num_classes, lower_shape)  # Random ints with scaled shape in num_classes range
noise = np.eye(num_classes)[noise] # sclaed shape * num_classes

# scale up noise labels
noise_up = skimage.transform.resize(noise,
                                    (shape[0], shape[1], num_classes),
                                    order=1, preserve_range=True,
                                    mode='constant') 

mask = np.floor(keep_prop + np.random.rand(*lower_shape))

mask_up = skimage.transform.resize(mask, (shape[0], shape[1], 1),
                                       order=1, preserve_range=True,
                                       mode='constant')

unary = mask_up * onehot_up + (1 - mask_up) * noise_up

/home/jimiolaniyan/anaconda2/envs/mldl/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


### Get predictions

In [37]:
from convcrf.convcrf import default_conf
import torch
import torch.nn as nn
from torch.autograd import Variable

FILTER_SIZE = 7

shape = image.shape[0:2]
config = default_conf
config['filter_size'] = FILTER_SIZE


# Pytorch uses C, H, W
# image = image.transpose(2,0,1) # [C, H, W]
image = image.transpose(2, 0, 1)  # shape: [3, hight, width]

# Add bactch dim
image = image.reshape([1, 3, shape[0], shape[1]]) 
# img_var = Variable(torch.Tensor(image)).cuda()

unary = unary.transpose(2, 0, 1)  # shape: [3, hight, width]
# Add batch dim
unary = unary.reshape([1, num_classes, shape[0], shape[1]])
# unary_var = Variable(torch.Tensor(unary)).cuda()

ValueError: axes don't match array

### Create args

In [137]:
from dotmap import DotMap

args = DotMap()
args.pyinn = False
args.nospeed = True 

In [ ]:
# prediction = do_crf_inference(image, unary, args)